In [28]:
import pandas as pd
import gradio as gr
import joblib as jb
import plotly.express as px
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

In [2]:
# load data and model
test_data = pd.read_csv("./data/test_data.csv")
model = jb.load("./model/model.pkl")

In [3]:
# split data
target = "Exited"
X = test_data.drop(columns=["RowNumber", "Exited"])
y = test_data[target]

# predict probabilities
prediction_probabilites = model.predict_proba(X)[:,1]

In [30]:
def show_metrics(probability_threshold, crc, cac):
    """
    Parameters
    ----------
    probability_threshold: float
        probability threshold for model prediction
    crc: int
        Customer retention cost
    cac: int
        Customer aquisition cost
    
    Returns
    -------
    figure: ConfusionMatrix
        Confusion matrix showing True values vs Predicted values
    evaluation metrics: float
        Accuracy, Precision, and Recall scores
    amounts: int
        costs associated with model predictions
    """

    # get predictions
    predictions = (prediction_probabilites >= probability_threshold).astype(int)

    # get confusion matrix
    matrix_array = confusion_matrix(y, predictions)

    # plot confusion matrix
    fig = px.imshow(
        matrix_array,
        labels=dict(x="Predicted Value", y="True Value"),
        x = ["False", "True"],
        y = ["False", "True"],
        text_auto=True,
        aspect="auto",
        color_continuous_scale=px.colors.sequential.gray
        )
    fig.update_xaxes(side="bottom")
    fig.update_layout(title_text="Confusion Matrix", title_x=0.5, coloraxis_showscale=False)

    # get metrics
    accuracy = accuracy_score(y, predictions).round(2)
    precision = precision_score(y, predictions).round(2)
    recall = recall_score(y, predictions).round(2)

    # get costs
    retention_cost = matrix_array.sum(axis=0)[1] * crc # crc cost with model
    aquisition_cost = matrix_array[1][0] * cac # cac with model
    total = retention_cost + aquisition_cost # total amount spent on customer aquisition and retention
    amount_saved = (matrix_array.sum(axis=1)[1] * cac) - total

    return fig, accuracy, precision, recall, retention_cost, aquisition_cost, total, amount_saved

In [31]:
# gradio blocks

with gr.Blocks() as demo:

    with gr.Row():
        pr_threshold = gr.Slider(minimum=0.0, maximum=1.0, value=0.5)

    with gr.Row():
        pr_confusion_matrix = gr.Plot(label="True Value vs Predicted Value")

    with gr.Row():
        # with gr.Column():
            accuracy = gr.Number(label="Accuracy Score")
            precision = gr.Number(label="Precision Score")
            recall = gr.Number(label="Recall Score")

    with gr.Row():
        crc = gr.Number(value=50, label="Customer Retention Cost (€)")
        cac = gr.Number(value=200, label="Customer Acquisiton Cost(€)")

    with gr.Row():
        total_crc = gr.Number(label="Customer Retention Cost(€)")
        total_cac = gr.Number(label="Customer Aquisition Cost(€)")
    with gr.Row():
        total_amount = gr.Number(label="Total Amount(€)")
        amount_saved = gr.Number(label="Amount Saved(€)")

    show_metrics_btn = gr.Button("Show Metrics")
    show_metrics_btn.click(
        fn=show_metrics,
        inputs=[pr_threshold, crc, cac],
        outputs=[pr_confusion_matrix, accuracy, precision, recall, total_crc, total_cac, total_amount, amount_saved]
        )

demo.launch()

Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7f549427dc30>, 'http://127.0.0.1:7871/', None)